#### ID 2080

```Count the number of unique users per day who logged in from both a mobile device and web. Output the date and the corresponding number of users.```

In [ ]:
%%sql
SELECT ml.date, COUNT(DISTINCT ml.user_id) AS cnt
FROM mobile_logs AS ml
         JOIN web_logs AS wl ON ml.user_id = wl.user_id AND ml.date = wl.date
GROUP BY ml.date

In [ ]:
df = pd.merge(mobile_logs, web_logs, how='inner', left_on=['user_id', 'date'], right_on=['user_id', 'date'])

df.groupby('date', as_index=False).agg(cnt=('user_id', 'nunique'))

#### ID 2083

```Count how many claims submitted in December 2021 are still pending. A claim is pending when it has neither an acceptance nor rejection date.```

In [ ]:
%%sql
SELECT COUNT(claim_id) AS n_claims
FROM cvs_claims
WHERE (EXTRACT(YEAR FROM date_submitted) = 2021 AND
       EXTRACT(MONTH FROM date_submitted) = 12)
  AND (date_accepted IS NULL AND date_rejected IS NULL)

In [ ]:
df = cvs_claims
df['year'] = df['date_submitted'].dt.year
df['month'] = df['date_submitted'].dt.month

df.query('year == 2021 & month == 12 & date_accepted.isnull() & date_rejected.isnull()')['claim_id'].count()

#### ID 2091

```For each video game player, find the latest date when they logged in.```

In [ ]:
%%sql
SELECT player_id, MAX(login_date) AS last_date
FROM players_logins
GROUP BY player_id

In [ ]:
df = players_logins
df.groupby('player_id', as_index=False).agg(max_date=('login_date', 'max'))

#### ID 2092

```You have been asked to find the top 3 merchants for each day with the highest number of orders on that day. In the event of a tie, multiple merchants may share the same spot, but each day at least one merchant must be in first, second, and third place. Your output should include the date in the format YYYY-MM-DD, the name of the merchant, and their place in the daily ranking.```

In [ ]:
%%sql
WITH cnt_orders_by_date_merchant AS (SELECT order_timestamp::DATE AS order_day,
                                            merchant_id,
                                            COUNT(id)             AS cnt_orders
                                     FROM order_details
                                     GROUP BY order_day, merchant_id),
     ranked_cnt_orders_by_date_merchant AS (SELECT order_day,
                                                   name,
                                                   DENSE_RANK()
                                                   OVER (PARTITION BY order_day ORDER BY cnt_orders DESC) AS ranking
                                            FROM cnt_orders_by_date_merchant
                                                     JOIN merchant_details AS md
                                                          ON cnt_orders_by_date_merchant.merchant_id = md.id)
SELECT order_day, name, ranking
FROM ranked_cnt_orders_by_date_merchant
WHERE ranking <= 3

In [ ]:
df['date'] = df['order_timestamp'].dt.strftime('%Y-%m-%d')

grouped = df.groupby(['date', 'merchant_id'], as_index=False).agg(n_orders=('id', 'count'))

grouped['ranking'] = grouped.groupby('date')['n_orders'].rank(method='dense', ascending=False)

grouped.query('ranking <= 3').merge(merchant_details, how='left', left_on='merchant_id', right_on='id')[['date', 'name', 'ranking']]

#### ID 2096

```Find the number of actions that ClassPass workers did for tasks completed in January 2022. The completed tasks are these rows in the asana_actions table with 'action_name' equal to CompleteTask. Note that each row in the dataset indicates how many actions of a certain type one user has performed in one day and the number of actions is stored in the 'num_actions' column. Output the ID of the user and a total number of actions they performed for tasks they completed. If a user from this company did not complete any tasks in the given period of time, you should still output their ID and the number 0 in the second column.```

In [ ]:
%%sql
SELECT aa.user_id,
       SUM(CASE
               WHEN action_name = 'CompleteTask' THEN num_actions
               ELSE 0 END) AS n_completed_tasks
FROM asana_actions aa
         JOIN asana_users au ON aa.user_id = au.user_id
WHERE DATE_PART('year', date) = 2022
  AND DATE_PART('month', date) = 1
  AND company = 'ClassPass'
GROUP BY aa.user_id

In [ ]:
df = pd.merge(asana_actions, asana_users, how='inner', on='user_id')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

df.query('company == "ClassPass" & month == 1 & year == 2022').groupby('user_id', as_index=False).agg(
    num_actions=('num_actions', lambda x: x[df['action_name'] == 'CompleteTask'].sum()))